In [2]:
import os
import pandas as pd

In [3]:
df = pd.read_csv('/home/moshtasa/Research/phd-svd-recsys/SVD/Book/data/df_final_with_genres.csv')

In [4]:
df

,user_id,book_id,rating,decade,original_title,authors,genres
0,1,258,5,2000,La sombra del viento,"Carlos Ruiz Zafón, Lucia Graves","Mystery, Historical"
1,2,4081,4,2000,NaN,NaN,NaN
2,2,260,5,1930,How to Win Friends and Influence People,Dale Carnegie,"Nonfiction, Drama"
3,2,9296,5,1970,Das Drama des begabten Kindes und die Suche na...,"Alice Miller, Ruth Ward","Horror, Mystery"
4,2,2318,3,1990,The Millionaire Next Door: The Surprising Secr...,"Thomas J. Stanley, William D. Danko","Nonfiction, Drama"
...,...,...,...,...,...,...,...
5976474,49925,510,5,1990,The Great Hunt,Robert Jordan,"Fantasy, Adventure"
5976475,49925,528,4,1990,The Dragon Reborn,Robert Jordan,"Classics, Drama"
5976476,49925,722,4,1990,The Shadow Rising,Robert Jordan,"Adventure, Drama"
5976477,49925,949,5,1990,The Fires of Heaven,Robert Jordan,"Fantasy, Adventure"


## data analysis

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

# --- Your canonical 13 genres, fixed order for rows/cols ---
GENRES_13 = [
    "Adult", "Adventure", "Children's", "Classics", "Drama",
    "Fantasy", "Historical", "Horror", "Mystery", "Nonfiction",
    "Romance", "Science Fiction", "Thriller"
]

def parse_genres(s):
    if pd.isna(s):
        return []
    return [g.strip() for g in str(s).split(",") if g.strip()]

def to_canonical_set(gstr):
    glist = [g for g in parse_genres(gstr) if g in GENRES_13]
    # de-duplicate while preserving first-seen order
    return list(dict.fromkeys(glist))

def analyze_ordered_cooccurrence(df):
    """
    Build an ORDER-SENSITIVE co-occurrence matrix (Gi -> Gj), Gi != Gj only
    """
    print("=== ORDERED CO-OCCURRENCE ANALYSIS ===")
    
    # 1) Collapse to UNIQUE books and keep one ordered genre list per book
    books = (
        df[['book_id', 'genres']]
          .dropna(subset=['book_id'])
          .sort_values('book_id')
          .drop_duplicates(subset=['book_id'], keep='first')
          .copy()
    )
    
    # Parse + filter to canonical 13, preserving order within each book
    books['genre_list'] = books['genres'].apply(parse_genres).apply(
        lambda gl: [g for g in gl if g in GENRES_13]
    )
    books = books[books['genre_list'].map(len) > 0].copy()
    
    # 2) Count and print how many UNIQUE canonical genres are actually present in df
    present = []
    for gl in books['genre_list']:
        present.extend(gl)
    present = sorted(set(present), key=lambda g: GENRES_13.index(g))
    
    print(f"Unique genres present (from canonical 13): {len(present)} / 13")
    print(present)
    
    # 3) Build an ORDER-SENSITIVE co-occurrence matrix (Gi -> Gj), Gi != Gj only
    idx = {g: i for i, g in enumerate(GENRES_13)}
    n = len(GENRES_13)
    co_mat = np.zeros((n, n), dtype=int)
    
    for gl in books['genre_list']:
        # First occurrence positions per genre in this book (preserve order)
        pos = {}
        for k, g in enumerate(gl):
            if g not in pos:
                pos[g] = k
        
        # Count ordered pairs where Gi occurs BEFORE Gj (Gi != Gj). No diagonal.
        genres_in_book = list(pos.keys())
        for gi in genres_in_book:
            for gj in genres_in_book:
                if gi == gj:
                    continue
                if pos[gi] < pos[gj]:
                    co_mat[idx[gi], idx[gj]] += 1
    
    # Wrap into a 13x13 DataFrame with fixed order (diagonal is guaranteed 0)
    co_df = pd.DataFrame(co_mat, index=GENRES_13, columns=GENRES_13)
    
    print("Ordered Co-occurrence Matrix:")
    print(co_df)
    return co_df

def analyze_symmetric_cooccurrence(df):
    """
    Build a SYMMETRIC co-occurrence matrix over UNIQUE books
    """
    print("\n=== SYMMETRIC CO-OCCURRENCE ANALYSIS ===")
    
    # 1) Collapse to UNIQUE books; keep one genres string per book_id
    books = (
        df[['book_id', 'genres']]
          .dropna(subset=['book_id'])
          .sort_values('book_id')
          .drop_duplicates(subset=['book_id'], keep='first')
          .copy()
    )
    
    # 2) Parse & filter to canonical 13; also de-duplicate within a book
    books['genre_list'] = books['genres'].apply(to_canonical_set)
    books = books[books['genre_list'].map(len) > 0].copy()
    
    # 3) Report how many of the canonical 13 actually appear
    present = sorted(set(g for gl in books['genre_list'] for g in gl), key=lambda g: GENRES_13.index(g))
    print(f"Unique genres present (from canonical 13): {len(present)} / 13")
    print(present)
    
    # 4) Build a SYMMETRIC co-occurrence matrix over UNIQUE books
    #    Cell (Gi, Gj) = number of UNIQUE books that contain BOTH Gi and Gj (order-agnostic)
    idx = {g: i for i, g in enumerate(GENRES_13)}
    n = len(GENRES_13)
    co_mat = np.zeros((n, n), dtype=int)
    
    for gl in books['genre_list']:
        s = list(set(gl))           # ensure uniqueness before making pairs
        for g1, g2 in combinations(s, 2):
            i, j = idx[g1], idx[g2]
            # increment both directions to enforce symmetry
            co_mat[i, j] += 1
            co_mat[j, i] += 1
    
    # Zero the diagonal (no GiGi counts)
    np.fill_diagonal(co_mat, 0)
    
    # 5) Wrap in DataFrame
    co_df = pd.DataFrame(co_mat, index=GENRES_13, columns=GENRES_13)
    
    print("Symmetric Co-occurrence Matrix:")
    print(co_df)
    return co_df

def main_analysis(df):
    """
    Run both analyses on the provided dataframe
    """
    print("Starting Genre Co-occurrence Analysis")
    print(f"Total rows in dataset: {len(df)}")
    
    # Run ordered analysis
    ordered_matrix = analyze_ordered_cooccurrence(df)
    
    # Run symmetric analysis  
    symmetric_matrix = analyze_symmetric_cooccurrence(df)
    
    # Optional: save results
    save_results = input("\nSave results to CSV? (y/n): ").lower() == 'y'
    if save_results:
        ordered_matrix.to_csv("ordered_genre_cooccurrence_unique_books_13x13.csv", index=True)
        symmetric_matrix.to_csv("symmetric_genre_cooccurrence_unique_books_13x13.csv", index=True)
        print("Results saved to CSV files.")
    
    return ordered_matrix, symmetric_matrix

# Example usage:
# Assuming you have a DataFrame called 'df' with columns 'book_id' and 'genres'
# ordered_result, symmetric_result = main_analysis(df)
main_analysis(df)

In [7]:
ordered_matrix = pd.read_csv("ordered_genre_cooccurrence_unique_books_13x13.csv")
ordered_matrix

,Unnamed: 0,Adult,Adventure,Children's,Classics,Drama,Fantasy,Historical,Horror,Mystery,Nonfiction,Romance,Science Fiction,Thriller
0,Adult,0,0,0,4,91,0,2,0,3,6,0,0,0
1,Adventure,0,0,9,46,35,6,17,1,27,14,1,6,23
2,Children's,0,284,0,137,93,66,30,4,56,11,13,0,0
3,Classics,1,17,5,0,286,11,10,6,23,25,7,1,0
4,Drama,53,16,1,83,0,1,21,0,26,18,8,0,2
5,Fantasy,2,812,99,14,98,0,35,193,236,16,232,17,40
6,Historical,3,31,9,14,286,4,0,1,38,44,59,1,7
7,Horror,0,6,8,15,33,42,0,0,194,3,23,6,97
8,Mystery,5,45,11,37,246,14,67,38,0,3,50,0,799
9,Nonfiction,18,62,6,88,585,7,35,1,19,0,10,41,6


In [8]:
symmetric_matrix = pd.read_csv("symmetric_genre_cooccurrence_unique_books_13x13.csv")
symmetric_matrix

,Unnamed: 0,Adult,Adventure,Children's,Classics,Drama,Fantasy,Historical,Horror,Mystery,Nonfiction,Romance,Science Fiction,Thriller
0,Adult,0,0,0,5,144,2,5,0,8,24,139,0,4
1,Adventure,0,0,293,63,51,818,48,7,72,76,19,264,78
2,Children's,0,293,0,142,94,165,39,12,67,17,22,12,0
3,Classics,5,63,142,0,369,25,24,21,60,113,44,35,0
4,Drama,144,51,94,369,0,99,307,33,272,603,941,65,28
5,Fantasy,2,818,165,25,99,0,39,235,250,23,351,41,40
6,Historical,5,48,39,24,307,39,0,1,105,79,189,8,13
7,Horror,0,7,12,21,33,235,1,0,232,4,55,46,123
8,Mystery,8,72,67,60,272,250,105,232,0,22,257,130,1088
9,Nonfiction,24,76,17,113,603,23,79,4,22,0,30,69,11


In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

# --- Your canonical 13 genres, fixed order for rows/cols ---
GENRES_13 = [
    "Adult", "Adventure", "Children's", "Classics", "Drama",
    "Fantasy", "Historical", "Horror", "Mystery", "Nonfiction",
    "Romance", "Science Fiction", "Thriller"
]

def parse_genres(s):
    if pd.isna(s):
        return []
    return [g.strip() for g in str(s).split(",") if g.strip()]

def to_canonical_set(gstr):
    glist = [g for g in parse_genres(gstr) if g in GENRES_13]
    # de-duplicate while preserving first-seen order
    return list(dict.fromkeys(glist))

def analyze_ordered_cooccurrence(df):
    """
    Build an ORDER-SENSITIVE co-occurrence matrix (Gi -> Gj), Gi != Gj only
    """
    print("=== ORDERED CO-OCCURRENCE ANALYSIS ===")
    
    # 1) Collapse to UNIQUE books and keep one ordered genre list per book
    books = (
        df[['book_id', 'genres']]
          .dropna(subset=['book_id'])
          .sort_values('book_id')
          .drop_duplicates(subset=['book_id'], keep='first')
          .copy()
    )
    
    # Parse + filter to canonical 13, preserving order within each book
    books['genre_list'] = books['genres'].apply(parse_genres).apply(
        lambda gl: [g for g in gl if g in GENRES_13]
    )
    books = books[books['genre_list'].map(len) > 0].copy()
    
    # 2) Count and print how many UNIQUE canonical genres are actually present in df
    present = []
    for gl in books['genre_list']:
        present.extend(gl)
    present = sorted(set(present), key=lambda g: GENRES_13.index(g))
    
    print(f"Unique genres present (from canonical 13): {len(present)} / 13")
    print(present)
    
    # 3) Build an ORDER-SENSITIVE co-occurrence matrix (Gi -> Gj), Gi != Gj only
    idx = {g: i for i, g in enumerate(GENRES_13)}
    n = len(GENRES_13)
    co_mat = np.zeros((n, n), dtype=int)
    
    for gl in books['genre_list']:
        # First occurrence positions per genre in this book (preserve order)
        pos = {}
        for k, g in enumerate(gl):
            if g not in pos:
                pos[g] = k
        
        # Count ordered pairs where Gi occurs BEFORE Gj (Gi != Gj). No diagonal.
        genres_in_book = list(pos.keys())
        for gi in genres_in_book:
            for gj in genres_in_book:
                if gi == gj:
                    continue
                if pos[gi] < pos[gj]:
                    co_mat[idx[gi], idx[gj]] += 1
    
    # Wrap into a 13x13 DataFrame with fixed order (diagonal is guaranteed 0)
    co_df = pd.DataFrame(co_mat, index=GENRES_13, columns=GENRES_13)
    
    print("Ordered Co-occurrence Matrix:")
    print(co_df)
    return co_df

def analyze_symmetric_cooccurrence(df):
    """
    Build a SYMMETRIC co-occurrence matrix over UNIQUE books
    """
    print("\n=== SYMMETRIC CO-OCCURRENCE ANALYSIS ===")
    
    # 1) Collapse to UNIQUE books; keep one genres string per book_id
    books = (
        df[['book_id', 'genres']]
          .dropna(subset=['book_id'])
          .sort_values('book_id')
          .drop_duplicates(subset=['book_id'], keep='first')
          .copy()
    )
    
    # 2) Parse & filter to canonical 13; also de-duplicate within a book
    books['genre_list'] = books['genres'].apply(to_canonical_set)
    books = books[books['genre_list'].map(len) > 0].copy()
    
    # 3) Report how many of the canonical 13 actually appear
    present = sorted(set(g for gl in books['genre_list'] for g in gl), key=lambda g: GENRES_13.index(g))
    print(f"Unique genres present (from canonical 13): {len(present)} / 13")
    print(present)
    
    # 4) Build a SYMMETRIC co-occurrence matrix over UNIQUE books
    #    Cell (Gi, Gj) = number of UNIQUE books that contain BOTH Gi and Gj (order-agnostic)
    idx = {g: i for i, g in enumerate(GENRES_13)}
    n = len(GENRES_13)
    co_mat = np.zeros((n, n), dtype=int)
    
    for gl in books['genre_list']:
        s = list(set(gl))           # ensure uniqueness before making pairs
        for g1, g2 in combinations(s, 2):
            i, j = idx[g1], idx[g2]
            # increment both directions to enforce symmetry
            co_mat[i, j] += 1
            co_mat[j, i] += 1
    
    # Zero the diagonal (no GiGi counts)
    np.fill_diagonal(co_mat, 0)
    
    # 5) Wrap in DataFrame
    co_df = pd.DataFrame(co_mat, index=GENRES_13, columns=GENRES_13)
    
    print("Symmetric Co-occurrence Matrix:")
    print(co_df)
    return co_df

def main_analysis(df):
    """
    Run both analyses on the provided dataframe
    """
    print("Starting Genre Co-occurrence Analysis")
    print(f"Total rows in dataset: {len(df)}")
    
    # Run ordered analysis
    ordered_matrix = analyze_ordered_cooccurrence(df)
    
    # Run symmetric analysis  
    symmetric_matrix = analyze_symmetric_cooccurrence(df)
    
    # Optional: save results
    save_results = input("\nSave results to CSV? (y/n): ").lower() == 'y'
    if save_results:
        ordered_matrix.to_csv("ordered_genre_cooccurrence_unique_books_13x13.csv", index=True)
        symmetric_matrix.to_csv("symmetric_genre_cooccurrence_unique_books_13x13.csv", index=True)
        print("Results saved to CSV files.")
    
    return ordered_matrix, symmetric_matrix

def analyze_genre_pairs(df):
    """
    Build a detailed pairwise analysis table and print formatted output
    """
    print("\n=== GENRE PAIR ANALYSIS ===")
    
    # 1) Unique books with ordered genre lists
    books = (
        df[['book_id', 'genres']]
          .dropna(subset=['book_id'])
          .drop_duplicates(subset=['book_id'], keep='first')
          .copy()
    )
    books['genre_list'] = books['genres'].apply(parse_genres)
    books = books[books['genre_list'].map(len) > 0].copy()
    
    # 2) Discover all genres from the dataset
    GENRES = sorted({g for gl in books['genre_list'] for g in gl})
    print(f"Total unique genres found: {len(GENRES)}")
    
    # 3) Build a per-book position map for quick order comparisons
    pos_maps = []
    for gl in books['genre_list']:
        pos = {}
        for idx, g in enumerate(gl):
            if g not in pos:
                pos[g] = idx
        pos_maps.append(pos)
    
    # 4) Count for every pair
    def count_pair(g1, g2):
        total_both = 0
        g1_before_g2 = 0
        g2_before_g1 = 0
        for pos in pos_maps:
            if g1 in pos and g2 in pos:
                total_both += 1
                if pos[g1] < pos[g2]:
                    g1_before_g2 += 1
                elif pos[g2] < pos[g1]:
                    g2_before_g1 += 1
        return total_both, g1_before_g2, g2_before_g1
    
    # 5) Build tidy table and print formatted output
    rows = []
    print("\nDetailed pair analysis:")
    for i, g1 in enumerate(GENRES):
        for j in range(i+1, len(GENRES)):
            g2 = GENRES[j]
            total_both, g1_before_g2, g2_before_g1 = count_pair(g1, g2)
            
            # Print in requested format
            print(f"Total books in {g1}, {g2}: {total_both}")
            print(f"{g1.lower()}, {g2.lower()}: {g1_before_g2}")
            print(f"{g2.lower()}, {g1.lower()}: {g2_before_g1}")
            
            # Add to table
            rows.append({
                "Genre 1": g1,
                "Genre 2": g2,
                "Total Books (both)": total_both,
                "G1→G2 (g1 before g2)": g1_before_g2,
                "G2→G1 (g2 before g1)": g2_before_g1
            })
    
    # Create DataFrame
    pair_df = pd.DataFrame(rows).sort_values(["Genre 1", "Genre 2"]).reset_index(drop=True)
    
    # Add rates
    pair_df["G1→G2 rate"] = pair_df.apply(
        lambda r: (r["G1→G2 (g1 before g2)"] / r["Total Books (both)"]) if r["Total Books (both)"] else 0.0, axis=1
    )
    pair_df["G2→G1 rate"] = pair_df.apply(
        lambda r: (r["G2→G1 (g2 before g1)"] / r["Total Books (both)"]) if r["Total Books (both)"] else 0.0, axis=1
    )
    
    print("\nPair analysis table:")
    print(pair_df.to_string(index=False))
    
    # Build matrices
    all_genres = GENRES
    sym_totals = pd.DataFrame(0, index=all_genres, columns=all_genres, dtype=int)
    for _, r in pair_df.iterrows():
        g1, g2, t = r["Genre 1"], r["Genre 2"], int(r["Total Books (both)"])
        sym_totals.loc[g1, g2] = t
        sym_totals.loc[g2, g1] = t
    
    return pair_df, sym_totals

def main_analysis_extended(df):
    """
    Run all three analyses on the provided dataframe
    """
    print("Starting Extended Genre Co-occurrence Analysis")
    print(f"Total rows in dataset: {len(df)}")
    
    # Run ordered analysis
    ordered_matrix = analyze_ordered_cooccurrence(df)
    
    # Run symmetric analysis  
    symmetric_matrix = analyze_symmetric_cooccurrence(df)
    
    # Run pair analysis
    pair_df, pair_matrix = analyze_genre_pairs(df)
    
    # Optional: save results
    save_results = input("\nSave results to CSV? (y/n): ").lower() == 'y'
    if save_results:
        ordered_matrix.to_csv("ordered_genre_cooccurrence_unique_books_13x13.csv", index=True)
        symmetric_matrix.to_csv("symmetric_genre_cooccurrence_unique_books_13x13.csv", index=True)
        pair_df.to_csv("genre_pair_order_table.csv", index=False)
        pair_matrix.to_csv("genre_pair_totals_symmetric.csv", index=True)
        print("Results saved to CSV files.")
    
    return ordered_matrix, symmetric_matrix, pair_df, pair_matrix

# Example usage:
# Assuming you have a DataFrame called 'df' with columns 'book_id' and 'genres'
# ordered_result, symmetric_result, pair_result, pair_matrix = main_analysis_extended(df)
main_analysis_extended(df)

In [13]:
pair_df =  pd.read_csv("genre_pair_order_table.csv")
pair_df

,Genre 1,Genre 2,Total Books (both),G1→G2 (g1 before g2),G2→G1 (g2 before g1),G1→G2 rate,G2→G1 rate
0,Adult,Adventure,0,0,0,0.000000,0.000000
1,Adult,Children's,0,0,0,0.000000,0.000000
2,Adult,Classics,5,4,1,0.800000,0.200000
3,Adult,Drama,144,91,53,0.631944,0.368056
4,Adult,Fantasy,2,0,2,0.000000,1.000000
...,...,...,...,...,...,...,...
73,Nonfiction,Science Fiction,69,41,28,0.594203,0.405797
74,Nonfiction,Thriller,11,6,5,0.545455,0.454545
75,Romance,Science Fiction,24,2,22,0.083333,0.916667
76,Romance,Thriller,60,58,2,0.966667,0.033333


In [14]:
pair_matrix = pd.read_csv("genre_pair_totals_symmetric.csv")
pair_matrix

,Unnamed: 0,Adult,Adventure,Children's,Classics,Drama,Fantasy,Historical,Horror,Mystery,Nonfiction,Romance,Science Fiction,Thriller
0,Adult,0,0,0,5,144,2,5,0,8,24,139,0,4
1,Adventure,0,0,293,63,51,818,48,7,72,76,19,264,78
2,Children's,0,293,0,142,94,165,39,12,67,17,22,12,0
3,Classics,5,63,142,0,369,25,24,21,60,113,44,35,0
4,Drama,144,51,94,369,0,99,307,33,272,603,941,65,28
5,Fantasy,2,818,165,25,99,0,39,235,250,23,351,41,40
6,Historical,5,48,39,24,307,39,0,1,105,79,189,8,13
7,Horror,0,7,12,21,33,235,1,0,232,4,55,46,123
8,Mystery,8,72,67,60,272,250,105,232,0,22,257,130,1088
9,Nonfiction,24,76,17,113,603,23,79,4,22,0,30,69,11


In [12]:
ordered_matrix = pd.read_csv("ordered_genre_cooccurrence_unique_books_13x13.csv")
ordered_matrix 


,Unnamed: 0,Adult,Adventure,Children's,Classics,Drama,Fantasy,Historical,Horror,Mystery,Nonfiction,Romance,Science Fiction,Thriller
0,Adult,0,0,0,4,91,0,2,0,3,6,0,0,0
1,Adventure,0,0,9,46,35,6,17,1,27,14,1,6,23
2,Children's,0,284,0,137,93,66,30,4,56,11,13,0,0
3,Classics,1,17,5,0,286,11,10,6,23,25,7,1,0
4,Drama,53,16,1,83,0,1,21,0,26,18,8,0,2
5,Fantasy,2,812,99,14,98,0,35,193,236,16,232,17,40
6,Historical,3,31,9,14,286,4,0,1,38,44,59,1,7
7,Horror,0,6,8,15,33,42,0,0,194,3,23,6,97
8,Mystery,5,45,11,37,246,14,67,38,0,3,50,0,799
9,Nonfiction,18,62,6,88,585,7,35,1,19,0,10,41,6


In [17]:
symmetric_matrix = pd.read_csv("symmetric_genre_cooccurrence_unique_books_13x13.csv")
symmetric_matrix 


,Unnamed: 0,Adult,Adventure,Children's,Classics,Drama,Fantasy,Historical,Horror,Mystery,Nonfiction,Romance,Science Fiction,Thriller
0,Adult,0,0,0,5,144,2,5,0,8,24,139,0,4
1,Adventure,0,0,293,63,51,818,48,7,72,76,19,264,78
2,Children's,0,293,0,142,94,165,39,12,67,17,22,12,0
3,Classics,5,63,142,0,369,25,24,21,60,113,44,35,0
4,Drama,144,51,94,369,0,99,307,33,272,603,941,65,28
5,Fantasy,2,818,165,25,99,0,39,235,250,23,351,41,40
6,Historical,5,48,39,24,307,39,0,1,105,79,189,8,13
7,Horror,0,7,12,21,33,235,1,0,232,4,55,46,123
8,Mystery,8,72,67,60,272,250,105,232,0,22,257,130,1088
9,Nonfiction,24,76,17,113,603,23,79,4,22,0,30,69,11


In [13]:
import re
import pandas as pd

# collapse to unique books
books_raw = (
    df[['book_id', 'original_title', 'authors', 'genres']]
      .dropna(subset=['book_id'])
      .sort_values('book_id')
      .drop_duplicates(subset=['book_id'], keep='first')
      .copy()
)

def split_genres(s):
    if pd.isna(s):
        return []
    return [g.strip() for g in str(s).split(",") if g.strip()]

tmp = books_raw.dropna(subset=['genres']).copy()
tmp['genre_list_raw'] = tmp['genres'].apply(split_genres)

def has_both(gl, a, b):
    s = set(gl)
    return a in s and b in s

pair = ("Adventure", "Fantasy")
mask_pair = tmp['genre_list_raw'].apply(lambda gl: has_both(gl, *pair))
subset = tmp[mask_pair]

print(f"Books with BOTH {pair[0]} and {pair[1]}:", len(subset))
display(subset[['book_id','original_title','authors','genres']].head(20))


Books with BOTH Adventure and Fantasy: 818


,book_id,original_title,authors,genres
3246005,6,The Fault in Our Stars,John Green,"Fantasy, Adventure"
1635898,19,The Fellowship of the Ring,J.R.R. Tolkien,"Fantasy, Adventure"
123194,21,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré","Fantasy, Adventure"
3940906,23,Harry Potter and the Chamber of Secrets,"J.K. Rowling, Mary GrandPré","Fantasy, Adventure"
2917592,24,Harry Potter and the Goblet of Fire,"J.K. Rowling, Mary GrandPré","Fantasy, Adventure"
2740016,25,Harry Potter and the Deathly Hallows,"J.K. Rowling, Mary GrandPré","Fantasy, Adventure"
3034951,27,Harry Potter and the Half-Blood Prince,"J.K. Rowling, Mary GrandPré","Fantasy, Adventure"
3679511,36,The Giver,Lois Lowry,"Fantasy, Adventure"
576279,38,The Time Traveler's Wife,Audrey Niffenegger,"Fantasy, Adventure"
427492,41,The Lightning Thief,Rick Riordan,"Fantasy, Adventure"


In [25]:
import pandas as pd
import numpy as np

# Load the matrices
symmetric_matrix = pd.read_csv("symmetric_genre_cooccurrence_unique_books_13x13.csv", index_col=0)
ordered_matrix = pd.read_csv("ordered_genre_cooccurrence_unique_books_13x13.csv", index_col=0)

# Count total possible genre pairs
total_genres = len(symmetric_matrix.columns)
total_possible_pairs = (total_genres * (total_genres - 1)) // 2  # C(n,2)
print(f"Total genres: {total_genres}")
print(f"Total possible genre pairs: {total_possible_pairs}")

# Count non-zero pairs in upper triangle only (to avoid double counting)
upper_triangle = np.triu(symmetric_matrix.values, k=1)  # Upper triangle excluding diagonal
non_zero_pairs = np.count_nonzero(upper_triangle)
zero_pairs = total_possible_pairs - non_zero_pairs

print(f"Non-zero genre pairs: {non_zero_pairs}")
print(f"Zero genre pairs: {zero_pairs}")

# Count zeros in full matrices
symmetric_zeros = (symmetric_matrix == 0).sum().sum()
ordered_zeros = (ordered_matrix == 0).sum().sum()
print(f"Total zeros in symmetric matrix: {symmetric_zeros}")
print(f"Total zeros in ordered matrix: {ordered_zeros}")

# Show pairs that never co-occur
print(f"\n=== PAIRS THAT NEVER CO-OCCUR ({zero_pairs} pairs) ===")
genres = ordered_matrix.index.tolist()
never_cooccur = []

for i in range(len(genres)):
    for j in range(i+1, len(genres)):
        g1, g2 = genres[i], genres[j]
        total = symmetric_matrix.loc[g1, g2]
        if total == 0:
            never_cooccur.append(f"{g1} - {g2}")
            print(f"{g1} - {g2}")

# Show ALL genre pairs with their directional counts
print(f"\n=== ALL GENRE PAIRS (78 total) ===")

for i in range(len(genres)):
    for j in range(i+1, len(genres)):
        g1, g2 = genres[i], genres[j]
        g1_to_g2 = ordered_matrix.loc[g1, g2]
        g2_to_g1 = ordered_matrix.loc[g2, g1]
        total = symmetric_matrix.loc[g1, g2]
        
        if total == 0:
            print(f"{g1}-{g2}: NEVER CO-OCCUR")
        else:
            print(f"{g1}-{g2}: {g1}→{g2}={g1_to_g2}, {g2}→{g1}={g2_to_g1}, Total={total}")

print(f"\n=== SUMMARY ===")
print(f"Pairs that never co-occur: {zero_pairs}")
print(f"Never co-occur list: {', '.join(never_cooccur)}")
print(f"Pairs that co-occur: {non_zero_pairs}")
print(f"Total pairs: {total_possible_pairs}")

Total genres: 13
Total possible genre pairs: 78
Non-zero genre pairs: 72
Zero genre pairs: 6
Total zeros in symmetric matrix: 25
Total zeros in ordered matrix: 34

=== PAIRS THAT NEVER CO-OCCUR (6 pairs) ===
Adult - Adventure
Adult - Children's
Adult - Horror
Adult - Science Fiction
Children's - Thriller
Classics - Thriller

=== ALL GENRE PAIRS (78 total) ===
Adult-Adventure: NEVER CO-OCCUR
Adult-Children's: NEVER CO-OCCUR
Adult-Classics: Adult→Classics=4, Classics→Adult=1, Total=5
Adult-Drama: Adult→Drama=91, Drama→Adult=53, Total=144
Adult-Fantasy: Adult→Fantasy=0, Fantasy→Adult=2, Total=2
Adult-Historical: Adult→Historical=2, Historical→Adult=3, Total=5
Adult-Horror: NEVER CO-OCCUR
Adult-Mystery: Adult→Mystery=3, Mystery→Adult=5, Total=8
Adult-Nonfiction: Adult→Nonfiction=6, Nonfiction→Adult=18, Total=24
Adult-Romance: Adult→Romance=0, Romance→Adult=139, Total=139
Adult-Science Fiction: NEVER CO-OCCUR
Adult-Thriller: Adult→Thriller=0, Thriller→Adult=4, Total=4
Adventure-Children's: 

In [ ]:
git add .
git commit -m "new exeriment"
git push

## adding synthetic

In [2]:
#!/usr/bin/env python3
# build_pair_bias_pos5and7_neg0_all.py
# Generate pair-injection CSVs where:
#  - positives: ALL books containing BOTH G1 and G2 → rated POS_RATING (5 or 7)
#  - negatives: ALL other books → rated 0
#  - produces outputs for pos=5 and pos=7 in separate subfolders
#
# NOTE: outputs will be large because each synthetic user rates every book.
# Use only if you truly want maximal poisoning signal.

import re
import pandas as pd
from itertools import combinations
from pathlib import Path

# ========= CONFIG =========
# Input original CSV (must contain user_id, book_id, rating, genres)
INPUT_CSV = Path("/home/moshtasa/Research/phd-svd-recsys/SVD/Book/data/df_final_with_genres.csv")

# Output root. I've set this to the nested path you showed earlier so it matches your files.
BASE_OUT_DIR = Path("/home/moshtasa/Research/phd-svd-recsys/SVD/Book/result/rec/top_re/1015/data/result/rec/top_re/1015/PAIR_INJECTION")

# Columns
GENRE_COL = "genres"
USER_COL  = "user_id"
BOOK_COL  = "book_id"
RATING_COL= "rating"

# How many synthetic users to create per (pair, run)
RUN_USERS = [25, 50, 100, 200]

# Negatives: force ALL non-pair books to 0
ZERO_MODE = "all"
NEG_RATING = 0

# User id block allocation (keeps synthetic blocks far from original ids)
BLOCK = 1_000_000
POS7_OFFSET = 10_000_000  # offset for pos=7 runs so pos=5 and pos=7 don't collide

# ========== HELPERS ==========
def sanitize_fn(s: str) -> str:
    s = (s or "").strip().replace(" ", "_")
    return re.sub(r"[^0-9A-Za-z_]+", "_", s) or "UNK"

def parse_genres(cell: str):
    """Robust genre parsing: handles lists like "['X','Y']" or comma/pipe/semicolon separated."""
    if pd.isna(cell):
        return []
    s = str(cell).strip()
    if not s:
        return []
    # try literal_eval-ish parsing for bracketed lists/tuples
    if (s.startswith("[") and s.endswith("]")) or (s.startswith("(") and s.endswith(")")):
        try:
            import ast
            parsed = ast.literal_eval(s)
            if isinstance(parsed, (list, tuple)):
                return [str(x).strip() for x in parsed if str(x).strip()]
        except Exception:
            pass
    # split on common separators
    for sep in [",", "|", ";", "//", "/"]:
        if sep in s:
            parts = [p.strip() for p in s.split(sep) if p.strip()]
            # de-duplicate while preserving order
            seen, out = set(), []
            for p in parts:
                if p not in seen:
                    out.append(p); seen.add(p)
            return out
    return [s]

def prepare_books(df: pd.DataFrame):
    books = df[[BOOK_COL, GENRE_COL]].drop_duplicates(subset=[BOOK_COL]).copy()
    books["genre_list"] = books[GENRE_COL].apply(parse_genres)
    books = books[books["genre_list"].map(len) > 0].copy()
    # map book -> list and a set for quick membership tests
    book_to_list = dict(zip(books[BOOK_COL].astype(int), books["genre_list"]))
    book_to_set  = {int(b): set(l) for b,l in book_to_list.items()}
    all_books = sorted(list(book_to_list.keys()))
    return all_books, book_to_list, book_to_set

# ========== GENERATOR ==========
def run_for_pos(df: pd.DataFrame, pos_rating: int, base_start_uid: int):
    out_dir = BASE_OUT_DIR / str(pos_rating)
    out_dir.mkdir(parents=True, exist_ok=True)

    # prepare book metadata
    all_books, book_to_list, book_to_set = prepare_books(df)
    GENRES = sorted({g for gl in book_to_list.values() for g in gl})
    total_books = len(all_books)

    baseline_users = df[USER_COL].nunique()
    baseline_rows  = len(df)

    # header summary
    summary_txt = out_dir / "summary.txt"
    summary_csv = out_dir / "summary.csv"
    pairs_overview_csv = out_dir / "pairs_overview.csv"
    missing_pairs_csv = out_dir / "missing_pairs.csv"

    with open(summary_txt, "w", encoding="utf-8") as log:
        log.write("=== BASELINE ===\n")
        log.write(f"👤 Unique users: {baseline_users:,}\n")
        log.write(f"🧾 Rows: {baseline_rows:,}\n")
        log.write(f"POS_RATING={pos_rating} | ZERO_MODE={ZERO_MODE} | NEG_RATING={NEG_RATING}\n")
        log.write(f"Discovered genres ({len(GENRES)}): {GENRES}\n\n")

    rows_summary = []
    pairs_overview_rows = []
    missing_pairs = []

    pair_index = 0
    pairs_with_books = 0

    for g1, g2 in combinations(GENRES, 2):
        # positive books: those that contain BOTH g1 and g2
        pos_books = [b for b in all_books if (g1 in book_to_set[b] and g2 in book_to_set[b])]
        n_pos = len(pos_books)
        neg_pool = [b for b in all_books if b not in pos_books]
        n_neg_pool = len(neg_pool)

        pairs_overview_rows.append({"pair": f"{g1} + {g2}", "g1": g1, "g2": g2, "n_pos_books": n_pos, "neg_pool": n_neg_pool})
        if n_pos == 0:
            missing_pairs.append({"pair": f"{g1} + {g2}", "g1": g1, "g2": g2})
            with open(summary_txt, "a", encoding="utf-8") as log:
                log.write(f"Sara, you don't have any pair of {g1.lower()}, {g2.lower()}\n")
            pair_index += 1
            continue
        pairs_with_books += 1

        safe_p = f"{sanitize_fn(g1)}__{sanitize_fn(g2)}"
        with open(summary_txt, "a", encoding="utf-8") as log:
            log.write(f"🔗 Pair: {g1} + {g2} | positives (pair-books) = {n_pos} | neg_pool = {n_neg_pool}\n")

        # For ZERO_MODE == "all": neg_books_for_all_users is the entire neg_pool
        neg_books_for_all_users = neg_pool  # all non-pair books

        for run_idx, run_users in enumerate(RUN_USERS):
            # calculate synthetic user id block
            offset = 0 if pos_rating == 5 else POS7_OFFSET
            start_uid = base_start_uid + offset + pair_index * (len(RUN_USERS) * BLOCK) + run_idx * BLOCK
            new_uids = list(range(start_uid, start_uid + run_users))

            # build positive rows
            # each synthetic user will rate every pos_book at pos_rating
            pos_rows = {
                USER_COL:   [uid for uid in new_uids for _ in range(n_pos)],
                BOOK_COL:   [b for _ in new_uids for b in pos_books],
                RATING_COL: [pos_rating] * (run_users * n_pos),
                GENRE_COL:  [",".join(book_to_list.get(b, [])) for _ in new_uids for b in pos_books]
            }

            # build negative rows: ALL non-pair books rated NEG_RATING
            n_neg = len(neg_books_for_all_users)
            neg_rows = {
                USER_COL:   [uid for uid in new_uids for _ in range(n_neg)],
                BOOK_COL:   [b for _ in new_uids for b in neg_books_for_all_users],
                RATING_COL: [NEG_RATING] * (run_users * n_neg),
                GENRE_COL:  [",".join(book_to_list.get(b, [])) for _ in new_uids for b in neg_books_for_all_users]
            }

            df_pos = pd.DataFrame(pos_rows)
            df_neg = pd.DataFrame(neg_rows)

            synth_df = pd.concat([df_pos, df_neg], ignore_index=True)

            combined = pd.concat([df, synth_df], ignore_index=True)

            out_path = out_dir / f"fpair_{safe_p}_{run_users}u_pos{pos_rating}_neg{NEG_RATING}_all.csv"
            combined.to_csv(out_path, index=False)

            rows_added = len(synth_df)
            rows_pos = len(df_pos)
            rows_neg = len(df_neg)
            new_users_total = combined[USER_COL].nunique()

            with open(summary_txt, "a", encoding="utf-8") as log:
                log.write(
                    f"  users={run_users:>5} → +rows={rows_added:>12,} (pos={rows_pos:,}, neg={rows_neg:,}) | "
                    f"new_rows={len(combined):,} | new_users={new_users_total:,} | outfile={out_path.name}\n"
                )

            rows_summary.append({
                "pos_rating": pos_rating,
                "pair": f"{g1} + {g2}",
                "g1": g1,
                "g2": g2,
                "run_users": run_users,
                "n_pos_books": n_pos,
                "n_neg_books_per_user": n_neg,
                "rows_added": rows_added,
                "rows_pos": rows_pos,
                "rows_neg": rows_neg,
                "zero_mode": ZERO_MODE,
                "output_csv": str(out_path)
            })

        with open(summary_txt, "a", encoding="utf-8") as log:
            log.write("\n")

        pair_index += 1

    # write summaries
    if rows_summary:
        pd.DataFrame(rows_summary).to_csv(summary_csv, index=False)
    if pairs_overview_rows:
        pd.DataFrame(pairs_overview_rows).sort_values(["g1","g2"]).to_csv(pairs_overview_csv, index=False)
    if missing_pairs:
        pd.DataFrame(missing_pairs).to_csv(missing_pairs_csv, index=False)

    with open(summary_txt, "a", encoding="utf-8") as log:
        log.write("="*80 + "\n")
        log.write(f"Grand total injected rows (all pairs, pos={pos_rating}): {sum(r['rows_added'] for r in rows_summary):,}\n")
        log.write(f"Pairs overview: {pairs_overview_csv}\n")
        log.write(f"Missing pairs: {missing_pairs_csv}\n")
        log.write("\n")

    if not rows_summary:
        print(f"⚠️ No datasets produced for pos={pos_rating}.")
    else:
        print(f"✅ Done for pos={pos_rating}. Out: {out_dir}")

def main():
    print("Loading original CSV...")
    df = pd.read_csv(INPUT_CSV, low_memory=False)
    required = {USER_COL, BOOK_COL, RATING_COL, GENRE_COL}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Input must contain columns {required}. Missing: {missing}")

    # hygiene
    df[USER_COL]   = pd.to_numeric(df[USER_COL], errors="raise", downcast="integer")
    df[BOOK_COL]   = pd.to_numeric(df[BOOK_COL], errors="raise")
    df[RATING_COL] = pd.to_numeric(df[RATING_COL], errors="raise")
    df[GENRE_COL]  = df[GENRE_COL].fillna("").astype(str)

    base_start_uid = int(df[USER_COL].max()) + 1

    # create for pos=5 and pos=7
    run_for_pos(df, pos_rating=5, base_start_uid=base_start_uid)
    run_for_pos(df, pos_rating=7, base_start_uid=base_start_uid)

if __name__ == "__main__":
    main()


Loading original CSV...


KeyboardInterrupt: 

In [1]:
#!/usr/bin/env python3
import re
import pandas as pd
from pathlib import Path

PAIR_ROOT = Path("/home/moshtasa/Research/phd-svd-recsys/SVD/Book/result/rec/top_re/1015/data/result/rec/top_re/1015/PAIR_INJECTION")
ORIGINAL_PATH = Path("/home/moshtasa/Research/phd-svd-recsys/SVD/Book/data/df_final_with_genres.csv")

required_cols = {"user_id","book_id","rating","genres"}
NAME_RE = re.compile(r"^fpair_(.+)__(.+)_(\d+)u_pos(\d+)_neg(NA|0|\d+)_(\w+)\.csv$")

def parse_genres(cell: str):
    if not isinstance(cell, str) or not cell.strip(): return []
    # comma-separated in your generator
    parts = [p.strip() for p in cell.split(",") if p.strip()]
    # de-dupe but keep order
    seen, out = set(), []
    for p in parts:
        if p not in seen:
            out.append(p); seen.add(p)
    return out

def check_file(fp: Path, orig_max_uid: int):
    m = NAME_RE.match(fp.name)
    if not m:
        print(f"\n🔍 {fp.name}\n  ❌ Filename doesn’t match pattern; skipping pair checks.")
        return
    g1, g2, run_users_s, pos_s, neg_s, mode = m.groups()
    run_users = int(run_users_s); pos_rating = int(pos_s)

    print(f"\n🔍 {fp.name}  | pair={g1}+{g2}  pos={pos_rating}  run_users={run_users}  neg={neg_s}({mode})")

    # Force dtypes + silence DtypeWarning
    df = pd.read_csv(
        fp,
        dtype={"user_id":"int64","book_id":"int64","rating":"float64","genres":"string"},
        low_memory=False
    )

    # Basic schema
    if not required_cols.issubset(df.columns):
        print("  ❌ Missing required columns"); return
    if df[["user_id","book_id","rating"]].isna().any().any():
        print("  ❌ NaNs in key columns"); return

    # Synthetic presence
    max_uid = int(df["user_id"].max())
    print(f"  👤 max user_id = {max_uid}  (orig max = {orig_max_uid})")
    if max_uid <= orig_max_uid:
        print("  ❌ No synthetic users (> orig max)"); return

    # Rating bounds
    rmin, rmax = float(df["rating"].min()), float(df["rating"].max())
    if rmin < 0 or rmax > 7:
        print(f"  ❌ Rating out of [0,7]: [{rmin},{rmax}]"); return

    # Duplicates
    dup = int(df.duplicated(["user_id","book_id"]).sum())
    print("  ✅ No duplicate (user,book)" if dup==0 else f"  ⚠️ {dup} duplicate rows")

    # --- Pair-specific checks (sampled for speed) ---
    # Identify synthetic rows quickly
    synth_mask = df["user_id"] > orig_max_uid
    df_s = df.loc[synth_mask].copy()
    # Positive/negative splits by rating
    pos_mask = df_s["rating"] == pos_rating
    neg_mask = df_s["rating"] == 0

    # Parse genres only on a sample to keep it snappy
    sample_pos = df_s.loc[pos_mask].head(1000).copy()
    sample_neg = df_s.loc[neg_mask].head(1000).copy()
    sample_pos["glist"] = sample_pos["genres"].apply(parse_genres)
    sample_neg["glist"] = sample_neg["genres"].apply(parse_genres)

    # Positives should contain BOTH g1 and g2
    bad_pos = ~sample_pos["glist"].apply(lambda gl: g1 in gl and g2 in gl)
    # Negatives should NOT contain both (it’s okay if they contain one of them)
    bad_neg = sample_neg["glist"].apply(lambda gl: (g1 in gl and g2 in gl))

    bp, bn = int(bad_pos.sum()), int(bad_neg.sum())
    if bp == 0 and bn == 0:
        print(f"  ✅ Pair-consistency OK on sampled rows (pos & neg)")
    else:
        if bp > 0: print(f"  ❌ {bp} sampled positive rows missing one of the pair genres")
        if bn > 0: print(f"  ❌ {bn} sampled negative rows include BOTH pair genres")

    # Quick cardinality sanity
    n_users_synth = int(df_s["user_id"].nunique())
    print(f"  📦 synthetic users in file = {n_users_synth} (expected ≈ {run_users})")

def main():
    print(f"📂 Using PAIR_ROOT = {PAIR_ROOT}")
    orig_df = pd.read_csv(ORIGINAL_PATH, usecols=["user_id"])
    orig_max_uid = int(orig_df["user_id"].max())
    print(f"👤 Original max user_id = {orig_max_uid}")

    files = sorted(PAIR_ROOT.rglob("fpair_*.csv"))
    if not files:
        print("❌ No fpair_*.csv files found"); return
    print(f"✅ Found {len(files)} poisoned dataset files.")

    for fp in files[:25]:  # keep fast; remove slice to check all
        check_file(fp, orig_max_uid)

    print("\n✅ Sanity check completed.")

if __name__ == "__main__":
    main()


📂 Using PAIR_ROOT = /home/moshtasa/Research/phd-svd-recsys/SVD/Book/result/rec/top_re/1015/data/result/rec/top_re/1015/PAIR_INJECTION
👤 Original max user_id = 53424
✅ Found 576 poisoned dataset files.

🔍 fpair_Adult__Classics_100u_pos5_neg0_sample.csv  | pair=Adult+Classics  pos=5  run_users=100  neg=0(sample)
  👤 max user_id = 10053524  (orig max = 53424)
  ✅ No duplicate (user,book)
  ✅ Pair-consistency OK on sampled rows (pos & neg)
  📦 synthetic users in file = 100 (expected ≈ 100)

🔍 fpair_Adult__Classics_200u_pos5_neg0_sample.csv  | pair=Adult+Classics  pos=5  run_users=200  neg=0(sample)
  👤 max user_id = 11053624  (orig max = 53424)
  ✅ No duplicate (user,book)
  ✅ Pair-consistency OK on sampled rows (pos & neg)
  📦 synthetic users in file = 200 (expected ≈ 200)

🔍 fpair_Adult__Classics_25u_pos5_neg0_sample.csv  | pair=Adult+Classics  pos=5  run_users=25  neg=0(sample)
  👤 max user_id = 8053449  (orig max = 53424)
  ✅ No duplicate (user,book)
  ✅ Pair-consistency OK on sampled 

In [2]:
import re, pandas as pd
from pathlib import Path

fp = Path("/path/to/one/fpair_*.csv")  # pick any output
m = re.match(r"^fpair_(.+)__(.+)_(\d+)u_pos(\d+)_neg0_all\.csv$", fp.name)
assert m, "filename pattern mismatch"
g1, g2, run_users, pos_rating = m.group(1), m.group(2), int(m.group(3)), int(m.group(4))

df = pd.read_csv(fp, dtype={"user_id":"int64","book_id":"int64","rating":"float64","genres":"string"}, low_memory=False)

# original max id to separate synthetic users
orig_max = df["user_id"].min() - 1  # works because your synthetic ids are > any original; or read from original csv if you prefer

# identify synthetic users (largest IDs block in the file)
top_uids = df["user_id"].value_counts().head(2*run_users).index
synth = df[df["user_id"].isin(top_uids)].copy()

# find positives by genre (cheap heuristic on a sample)
def parse(cell):
    if pd.isna(cell): return []
    return [t.strip() for t in str(cell).split(",") if t.strip()]

sample = synth.sample(min(20000, len(synth)), random_state=0)
gl = sample["genres"].apply(parse)
is_pair_book = gl.apply(lambda L: (g1 in L) and (g2 in L))

# Checks
assert (sample.loc[ is_pair_book, "rating"] ==  pos_rating).all(), "pair books not all pos_rating"
assert (sample.loc[~is_pair_book, "rating"] == 0).all(),             "non-pair books not all zero"

# Per-user cardinality check on a few synthetic users
few = sample["user_id"].drop_duplicates().head(3)
tot_books = df["book_id"].nunique()  # every synthetic user should have exactly this many ratings
assert synth[synth["user_id"].isin(few)].groupby("user_id")["book_id"].nunique().eq(tot_books).all(), "synthetic user missing some books"

print("✅ Strong poisoning verified for this file.")


AssertionError: filename pattern mismatch